In [98]:
using DifferentialEquations
using Plots
using Printf
using LaTeXStrings

In [99]:
# Define the simple pendulum ODE
function pendulum!(du, u, p, t)
    θ, ω = u
    du[1] = ω               # dθ/dt = ω
    du[2] = -sin(θ)         # dω/dt = -sin(θ)
end

pendulum! (generic function with 1 method)

In [100]:
# Parameters
n_pendula = 5000            # Number of pendula
tspan = (0.0, 100.0)        # Time span
fps = 30                   # Frames per second for animation
total_frames = 300;         # Total frames in animation

In [101]:
# Create initial conditions that thoroughly sample phase space
θ0 = range(-π, π, length=ceil(Int, sqrt(n_pendula)))
ω0 = range(-2.0, 2.0, length=ceil(Int, sqrt(n_pendula)))

# Create all possible combinations and flatten
all_conditions = vec([[θ, ω] for θ in θ0, ω in ω0])

# Alternate between beginning and end to mix positive/negative ω points
n_total = length(all_conditions)
n_use = min(n_total, n_pendula)
indices = Int[]
i, j = 1, n_total

while length(indices) < n_use
    if i <= j
        push!(indices, i)
        i += 1
    end
    if length(indices) < n_use && j >= i
        push!(indices, j)
        j -= 1
    end
end

initial_conditions = all_conditions[indices][1:n_pendula];

In [102]:
# Set up the ODE problems
problems = [ODEProblem(pendulum!, ic, tspan) for ic in initial_conditions]

# Solve all ODEs with progress bar
println("Solving ODEs...")
solutions = [solve(prob, Tsit5(), saveat=range(tspan[1], tspan[2], length=total_frames)) for prob in problems];

Solving ODEs...


In [103]:
# Helper function for meshgrid (similar to MATLAB's meshgrid)
function meshgrid(x, y)
    X = [i for i in x, j in y]
    Y = [j for i in x, j in y]
    return X, Y
end

meshgrid (generic function with 1 method)

In [104]:
# Prepare the animation
println("Creating animation...")
Plots.theme(:dao)

# Create a color gradient based on initial ω values
initial_ω = [ic[2] for ic in initial_conditions]
colors = cgrad(:viridis)  # Choose any colormap you like (:viridis, :plasma, :inferno, etc.)

# Normalize initial ω values to [0,1] for colormap
ω_min, ω_max = extrema(initial_ω)
ω_norm = @. (initial_ω - ω_min) / (ω_max - ω_min)

# Assign colors based on initial ω
# point_colors = [colors[ω_norm[i]] for i in 1:n_pendula]

# Alternative: Binary coloring (positive/negative initial ω)
point_colors = [ω > 0 ? :blue : :red for ω in initial_ω]

anim = @animate for i in 1:total_frames
    t = (i - 1) / (total_frames - 1) * tspan[2]

    # Extract current state of all pendula at this frame
    θs = [sol[1, i] for sol in solutions]
    ωs = [sol[2, i] for sol in solutions]

    # Wrap θ to [-π, π] for plotting
    θs_wrapped = mod.(θs .+ π, 2π) .- π

    # Create phase space plot
    p = scatter(θs_wrapped, ωs,
        xlims=(-π, π), ylims=(-2.5, 2.5),
        xlabel="θ", ylabel=L"\dot{\theta}",
        title="Pendula Phase Space Mixing (t =$(@sprintf("%0.2f", t)))",
        legend=false, 
        markersize=6,
        alpha=0.7,
        markercolor=point_colors,  # Use our color assignments
        size=(800, 600),
        titlefontsize=20,
        tickfontsize=12,
        legendfontsize=10,
        yguidefontsize=15,
        xguidefontsize=15)

    # Add multiple energy contours
    θ_range = range(-π, π, length=500)
    for E in [0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
        ω_pos = @. sqrt(max(0, 2*(E + cos(θ_range))))
        ω_neg = @. -sqrt(max(0, 2*(E + cos(θ_range))))
        valid = 2*(E .+ cos.(θ_range)) .≥ 0
        plot!(θ_range[valid], ω_pos[valid], 
              color=:black, linestyle=:dash, linewidth=2, label="")
        plot!(θ_range[valid], ω_neg[valid], 
              color=:black, linestyle=:dash, linewidth=2, label="")
    end

    # Highlight the main separatrix
    main_sep_ω = 2 .* cos.(θ_range ./ 2)
    plot!(θ_range, main_sep_ω, color=:black, linewidth=2, label="Main Separatrix")
    plot!(θ_range, -main_sep_ω, color=:black, linewidth=2, label="")
    
    p
end

Creating animation...


Animation("/tmp/jl_pJJRgY", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000291.png", "000292.png", "000293.png", "000294.png", "000295.png", "000296.png", "000297.png", "000298.png", "000299.png", "000300.png"])

In [105]:
# Save the animation
mp4(anim, "pendulum_phase_mixing.mp4", fps=fps)
println("Animation saved to pendulum_phase_mixing.mp4")

Animation saved to pendulum_phase_mixing.mp4


[ Info: Saved animation to /home/belster/JuliaFiles/Landau/pendulum_phase_mixing.mp4
